Plan for steps in this processing code
1. Define a base grid
2. Load all the tif files - know how
4. Load the shp files - know how
5. Project the shp files to tif - at the resolution / bounds, etc with the base grid - know how
6. Generate the train deposit / occurence tif files - know how
6. Unify all the tif data - know how

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import rasterio
import rasterio.mask

from s2sphere import CellId


import utilities as utils

DATA_DIR = "data/LAWLEY22-RAW/geophysics/"

In [2]:
tifs, shps = utils.get_input_var_files("Australia")

Loads the raster data

In [3]:
rasters = utils.load_rasters(tifs, rasters_path=DATA_DIR, verbosity=1)

Loading GeophysicsMagRTP_HGMDeepSources_Australia: 100%|██████████| 12/12 [00:00<00:00, 459.77it/s]

-------- GeophysicsLAB_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.95, bottom=-43.65, right=153.65, top=-10.049999999999997),Size:(336, 407),Resolution:(0.1, 0.1)


-------- GeophysicsMoho_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.95, bottom=-43.65, right=153.65, top=-9.349999999999994),Size:(343, 407),Resolution:(0.1, 0.1)


-------- GeophysicsSatelliteGravity_ShapeIndex_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=113.0, bottom=-43.6, right=153.6, top=-10.600000000000001),Size:(165, 203),Resolution:(0.2, 0.2)


-------- GeophysicsGravity_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.91983

Loads rasters of the vector data if available; otherwise generates them

In [4]:
try:
    rasters += utils.load_rasters(shps, rasters_path=DATA_DIR, verbosity=1)
except rasterio.RasterioIOError:
    base_raster = rasters[-1] # defaults to intermediate resolution raster
    vectors = utils.load_vectors(shps, vectors_path=DATA_DIR, verbosity=0)
    pbar = tqdm(zip(shps, vectors))
    for shp, vector in pbar:
        pbar.set_description(f"Processing {shp}")
        utils.proximity_raster_of_vector_points(base_raster, shp, vector)
    rasters += utils.load_rasters(shps, rasters_path=DATA_DIR, verbosity=1)

Loading DeepMagSources_Worms_Australia: 100%|██████████| 4/4 [00:00<00:00, 587.38it/s]

-------- ShallowGravitySources_Worms_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.9185, bottom=-43.63974, right=153.63126, top=-10.04778000000001),Size:(4057, 4917),Resolution:(0.00828, 0.00828)


-------- DeepGravitySources_Worms_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.9185, bottom=-43.63974, right=153.63126, top=-10.04778000000001),Size:(4057, 4917),Resolution:(0.00828, 0.00828)


-------- ShallowMagSources_Worms_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordinate reference system:
EPSG:4326


Bounds:BoundingBox(left=112.9185, bottom=-43.63974, right=153.63126, top=-10.04778000000001),Size:(4057, 4917),Resolution:(0.00828, 0.00828)


-------- DeepMagSources_Worms_Australia raster details --------

Raster bands and dtypes:
{1: 'float32'}


Coordi

Loads the base grid for all data if available; otherwise generates it

In [5]:
grid_cell_file = f"{DATA_DIR}s2_grid_aus.npy"
try:
    grid_cell_ids = np.load(grid_cell_file)
except OSError:
    grid_bounds = utils.compute_bounds(rasters, verbosity=1)
    grid_cell_ids = [grid_cell.id() for grid_cell in utils.region_of_cellids(grid_bounds, s2_level=12)]
    np.save(grid_cell_file, np.asarray(grid_cell_ids, dtype=np.uint64))

Initialize the datacube

In [6]:
datacube = utils.init_datacube({"s2_cell_id": grid_cell_ids}, ["s2_cell_center", "s2_cell_poly"] + tifs + shps, verbosity=1)

Populate the datacube

In [17]:
for row_idx, row in tqdm(datacube.iterrows(),total=datacube.shape[0]):
    s2cell = CellId(int(row["s2_cell_id"]))
    row["s2_cell_center"] = utils.s2_cell_center(s2cell)
    row["s2_cell_poly"] = utils.s2_cell_polygon(s2cell)
    for col, raster in zip(tifs + shps, rasters):
        try:
            masked, _ = rasterio.mask.mask(raster, [row["s2_cell_poly"]], crop=True)
            if (raster.nodata == masked).all(): continue
            row[col] = np.mean(masked[raster.nodata != masked])
        except ValueError:
            continue

  0%|          | 0/3616977 [00:00<?, ?it/s]

  5%|▍         | 165487/3616977 [47:10<15:13:06, 63.00it/s]